In [19]:
import pandas as pd
import numpy as np
df = pd.read_csv('questions_eval_full.csv')
df

,question,ground_truth,retrieved_chunks,score,correct_rank
0,Quản lý rủi ro trong quản lý thuế được áp dụng...,9,"[9, 8, 12, 14, 16]",0.7012,1
1,Việc hoàn thuế được thực hiện trong trường hợp...,4,"[1, 4, 6, 7, 10]",0.8124,2
2,Mã số thuế được cấp và sử dụng ra sao?,35,"[32, 35, 36, 26, 28]",0.8954,2
3,"Theo quy định, các khoản thu nào được coi là t...",3,"[3, 4, 5, 6, 11]",0.9303,1
4,"Theo quy định, cơ quan nào chịu trách nhiệm hư...",19,"[12, 16, 19, 27, 28]",0.8874,3
...,...,...,...,...,...
145,Người nộp thuế có những quyền gì theo luật? Tr...,16,"[6, 11, 16, 17, 22]",0.7884,3
146,Ai là đối tượng áp dụng theo Luật Quản lý thuế...,2,"[2, 4, 6, 10, 11]",0.7475,1
147,"Theo quy định, trách nhiệm của người nộp thuế ...",17,"[13, 14, 16, 17, 26]",0.7244,4
148,Khi nào người nộp thuế được cấp giấy chứng nhậ...,34,"[34, 37, 26, 28, 30]",0.8942,1


In [20]:
df = df.rename(columns={"ground_truth": "expected_answer"})
df_2column = df[['question', 'expected_answer']]
df_2column

,question,expected_answer
0,Quản lý rủi ro trong quản lý thuế được áp dụng...,9
1,Việc hoàn thuế được thực hiện trong trường hợp...,4
2,Mã số thuế được cấp và sử dụng ra sao?,35
3,"Theo quy định, các khoản thu nào được coi là t...",3
4,"Theo quy định, cơ quan nào chịu trách nhiệm hư...",19
...,...,...
145,Người nộp thuế có những quyền gì theo luật? Tr...,16
146,Ai là đối tượng áp dụng theo Luật Quản lý thuế...,2
147,"Theo quy định, trách nhiệm của người nộp thuế ...",17
148,Khi nào người nộp thuế được cấp giấy chứng nhậ...,34


In [29]:
df3 = pd.read_json("test_chunks2.json")
df3['chunk_id']

0        1
1        2
2        3
3        4
4        5
      ... 
147    148
148    149
149    150
150    151
151    152
Name: chunk_id, Length: 152, dtype: int64

In [ ]:
import requests
result = []
for _, row in df.iterrows():
    query = row["question"]
    expected_chunk_id = row["expected_answer"]


    try:
        res = requests.get("http://localhost:8000/search/hybrid/", params={
            "query": query,
            "top_k": 5,
            "alpha": 0.5
        })
        res.raise_for_status()
        data = res.json()
        converted_data = pd.DataFrame(data)
        add_row = {"query": query,
                    "expected_answer": converted_data[converted_data['chunk_id'] == expected_chunk_id]['content'].values() if expected_chunk_id in converted_data['chunk_id'].values else None,
                    "expected_chunk_id": converted_data['chunk_id'].values.tolist(),
                    "label": 1 if expected_chunk_id in converted_data['chunk_id'].values else 0
                }
        result.append(add_row)
    except Exception as e:
        print(f"Lỗi với câu hỏi: {query} | {e}")

    

In [82]:
dp=pd.DataFrame(result)
dp['expected_answer']

0                                                   None
1                                                   None
2      [## Điều 35. Sử dụng mã số thuế\n\n1. Người nộ...
3      [## Điều 3. Giải thích từ ngữ\n\nTrong Luật nà...
4                                                   None
                             ...                        
145                                                 None
146                                                 None
147    [## Điều 17. Trách nhiệm của người nộp thuế\n\...
148    [## Điều 34. Cấp giấy chứng nhận đăng ký thuế\...
149                                                 None
Name: expected_answer, Length: 150, dtype: object

In [83]:
dp[dp['label'] == 1].to_csv('result_true.csv', index=False)
dp[dp['label'] == 0].to_csv('result_false.csv', index=False)

# Tỷ lệ tìm kiếm đúng trên tập dữ liệu câu hỏi tổng 150 câu

In [84]:
cout_true = dp['label'].value_counts()
print(f"Số lượng câu hỏi đúng: {cout_true[1]}")
print(f"Số lượng câu hỏi sai: {cout_true[0]}")
print(f"Tỷ lệ đúng: {cout_true[1] / (cout_true[0] + cout_true[1]) * 100:.2f}%")

Số lượng câu hỏi đúng: 86
Số lượng câu hỏi sai: 64
Tỷ lệ đúng: 57.33%


In [85]:
tt = pd.read_csv("result_true.csv")
tt

,query,expected_answer,expected_chunk_id,label
0,Mã số thuế được cấp và sử dụng ra sao?,['## Điều 35. Sử dụng mã số thuế\n\n1. Người n...,"[35, 6, 30, 3, 91]",1
1,"Theo quy định, các khoản thu nào được coi là t...",['## Điều 3. Giải thích từ ngữ\n\nTrong Luật n...,"[3, 15, 134, 18, 42]",1
2,"Theo quy định, mã số thuế được cấp và sử dụng ...",['## Điều 35. Sử dụng mã số thuế\n\n1. Người n...,"[3, 30, 6, 17, 35]",1
3,Các khoản thu nào được coi là thuộc ngân sách ...,['## Điều 3. Giải thích từ ngữ\n\nTrong Luật n...,"[15, 42, 134, 27, 3]",1
4,"Theo quy định, quyền của người nộp thuế khi bị...",['## Điều 16. Quyền của người nộp thuế\n\n1. Đ...,"[15, 16, 17, 50, 18]",1
...,...,...,...,...
81,Mã số thuế được cấp và sử dụng ra sao? Trong t...,['## Điều 35. Sử dụng mã số thuế\n\n1. Người n...,"[3, 35, 39, 30, 110]",1
82,Luật Quản lý thuế quy định phạm vi điều chỉnh ...,['## Điều 1. Phạm vi điều chỉnh\nLUẬT\n\nQUẢN ...,"[1, 122, 120, 116, 3]",1
83,Người nộp thuế phải làm gì khi thay đổi thông ...,['## Điều 36. Thông báo thay đổi thông tin đăn...,"[38, 30, 36, 60, 35]",1
84,"Theo quy định, trách nhiệm của người nộp thuế ...",['## Điều 17. Trách nhiệm của người nộp thuế\n...,"[17, 104, 19, 130, 118]",1


In [ ]:
import pandas as pd
import requests
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

df = pd.read_csv("result_true.csv")

predicted = []
true_labels = []

def is_semantic_match(expected, answer, threshold=0.75):
    """So sánh ngữ nghĩa giữa 2 câu, trả về True nếu đủ tương đồng"""
    if not expected or not answer:
        return False
    embedding_expected = model.encode(expected, convert_to_tensor=True)
    embedding_answer = model.encode(answer, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(embedding_expected, embedding_answer).item()
    return similarity_score >= threshold

for _, row in df.iterrows():
    query = row['query']
    expected = row['expected_answer']

    try:
        res = requests.get("http://localhost:8000/chat/", params={
            "query": query,
            "mode": "hybrid",  
            "top_k": 5,
            "alpha": 0.5
        })

        if res.status_code != 200:
            print(f"Lỗi truy vấn API: {query}")
            continue

        result = res.json()
        chatbot_answer = result["answer"]

        match = is_semantic_match(expected, chatbot_answer)

        predicted.append(int(match))
        true_labels.append(1)  

    except Exception as e:
        print(f" Lỗi xử lý câu hỏi '{query}': {e}")




KeyboardInterrupt: 

In [109]:
print(predicted[:43])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0]


# Đánh giá câu trả lời của chat bot thông qua 86 câu đã truy vấn đúng 
(Do  Gemini Free Tier chỉ giới free có 50 request ngày lên chỉ lấy đến 43 câu đúng)
Sử dụng model paraphrase-multilingual-MiniLM-L12-v2 để encode hai câu.

Tính cosine similarity giữa 2 embedding.

Trả về True nếu similarity ≥ threshold (mặc định 0.75).

👉 Tại sao dùng cosine similarity?
Vì embedding nằm trong không gian vector – cosine similarity cho biết hai vector gần nhau về hướng → câu có cùng nghĩa.

In [111]:
acc = accuracy_score(true_labels[:43], predicted[:43])
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted, average='binary')

print(f"Đánh giá chatbot:")
print(f"Accuracy:  {acc:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1-score:  {f1:.2f}")

Đánh giá chatbot:
Accuracy:  0.77
Precision: 1.00
Recall:    0.49
F1-score:  0.65
